# Titanic (catboost classifier)
* Notebook:
* kaggle: https://www.kaggle.com/c/titanic
* catboost: https://catboost.ai/


In [29]:
import math
from pprint import pprint
import json

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact

from IPython.display import JSON
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
print(f'pandas  version: {pd.__version__}')
print(f'numpy   version: {np.__version__}')
print(f'seaborn version: {sns.__version__}')

pandas  version: 1.1.5
numpy   version: 1.19.5
seaborn version: 0.11.1


data
Variable	Definition	Key
Survived	Survival	0 = No, 1 = Yes
Pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
Sex	Sex	
Age	Age in years	
SibSp	# of siblings / spouses aboard the Titanic	
Sibling = brother, sister, stepbrother, stepsister	
Spouse = husband, wife (mistresses and fiancés were ignored)	
Parch	# of parents / children aboard the Titanic	
Parent = mother, father	
Child = daughter, son, stepdaughter, stepson	
Some children travelled only with a nanny, therefore parch=0 for them.	
Ticket	Ticket number	
Fare	Passenger fare	
Cabin	Cabin number	
Embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [32]:
df=pd.read_csv('https://github.com/prasertcbs/basic-dataset/raw/master/titanic_train.csv')
df.sample(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
480,481,0,3,"Goodwin, Master. Harold Victor",male,9.0,5,2,CA 2144,46.9000,NaN,S
731,732,0,3,"Hassan, Mr. Houssein G N",male,11.0,0,0,2699,18.7875,NaN,C
757,758,0,2,"Bailey, Mr. Percy Andrew",male,18.0,0,0,29108,11.5000,NaN,S
82,83,1,3,"McDermott, Miss. Brigdet Delia",female,NaN,0,0,330932,7.7875,NaN,Q
248,249,1,1,"Beckwith, Mr. Richard Leonard",male,37.0,1,1,11751,52.5542,D35,S
180,181,0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.5500,NaN,S
60,61,0,3,"Sirayanian, Mr. Orsen",male,22.0,0,0,2669,7.2292,NaN,C
575,576,0,3,"Patchett, Mr. George",male,19.0,0,0,358585,14.5000,NaN,S
104,105,0,3,"Gustafsson, Mr. Anders Vilhelm",male,37.0,2,0,3101276,7.9250,NaN,S
50,51,0,3,"Panula, Master. Juha Niilo",male,7.0,4,1,3101295,39.6875,NaN,S


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [34]:
# df['Pclass']=df['Pclass'].map({1: '1st', 2: '2nd', 3: '3rd'})

In [35]:
# df['Embarked']=df['Embarked'].fillna('')

In [36]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## **model**

In [37]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, precision_recall_fscore_support, f1_score

In [38]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [39]:
feature_cols=[
    'Pclass', 
    'Sex', 
    'Age', 'SibSp', 'Parch', 
]
target_col='Survived'
# cat_features=[0, 1]
cat_features=['Pclass', 
              'Sex', 
              ]
X=df[feature_cols]
y=df[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=1, 
                                                    stratify=y)

# **catboost**
* catboost doc: https://catboost.ai/docs/concepts/about.html
* how to install catboost

* pip: pip install -U catboost or
* conda: conda install -c conda-forge catboost

In [40]:
!pip install -U catboost

Requirement already up-to-date: catboost in /usr/local/lib/python3.7/dist-packages (0.25.1)


In [41]:
import catboost as cb
print(f'catboost version = {cb.__version__}')

catboost version = 0.25.1


In [42]:
params={
    'iterations': 100, # max number of weak learners
    'max_depth': 4, 
#     'auto_class_weights': 'Balanced'
}
clf = cb.CatBoostClassifier(**params)

# clf = cb.CatBoostClassifier()params={
    'iterations': 100, # max number of weak learners
    'max_depth': 4, 
#     'auto_class_weights': 'Balanced'
}
clf = cb.CatBoostClassifier(**params)

# clf = cb.CatBoostClassifier()

IndentationError: ignored

In [ ]:
fit_params={
    'cat_features': cat_features,
    'early_stopping_rounds': 10, 
    'eval_set': [(X_test, y_test)],
    'verbose': 10,
    'plot': True
           }
clf.fit(X_train, y_train, **fit_params)
# clf.fit(X_train, y_train)

In [ ]:
clf.get_params()

In [ ]:
clf.get_all_params()

In [ ]:
clf.score(X_train, y_train) # accuracy score

In [ ]:
clf.score(X_test, y_test) # accuracy score

In [ ]:
clf.predict(X_test[:5])

In [ ]:
clf.predict_proba(X_test[:5])

# confusion matrix
$$
\begin{bmatrix}
C_{0,0} &amp; C_{0,1} \\
C_{1,0} &amp; C_{1,1}
\end{bmatrix}
=
\begin{bmatrix}
tn&amp;fp \\
fn&amp;tp
\end{bmatrix}
$$
predicted false	predicted true
actual false	tn	fp
actual true	fn	tp


In [ ]:
cm=confusion_matrix(y_test, clf.predict(X_test))
cm

In [ ]:
plt.rcParams['font.size']=13
plot_confusion_matrix(clf, X_test, y_test, cmap='Oranges', 
                      display_labels=['dead', 'survived']);

$$
\begin{align}
\text {Accuracy}&amp;={\frac {{TP+TN} }{TP + TN +FP + FN} }\\ \\
\text {Precision}&amp;={\frac {TP}{TP+FP}}\\ \\
\text {Recall, True Positive Rate (TPR), sensitivity}&amp;={\frac {TP}{TP+FN}}\\ \\
\text {False Positive Rate (FPR), miss rate}&amp;={\frac {FP}{FP+TN}}\\ \\
F1&amp;=2 \times {\frac {precision \times recall} {precision + recall}}\\ \\
Recall_0&amp;={\frac {TN}{TN+FP}}\\ \\
Recall_1&amp;={\frac {TP}{TP+FN}}\\ \\
Precision_0&amp;={\frac {TN}{TN+FN}}\\ \\
Precision_1&amp;={\frac {TP}{TP+FP}}
\end{align}
$$

In [43]:
print(classification_report(y_train, clf.predict(X_train)))

NameError: ignored

In [44]:
print(classification_report(y_test, clf.predict(X_test)))

NameError: ignored

In [45]:
print(classification_report(y_test, clf.predict(X_test)))

NameError: ignored

In [46]:
clf.feature_importances_

NameError: ignored

In [47]:
X.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')

In [48]:
feature_importances=pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
feature_importances

NameError: ignored

In [49]:
feature_importances.sum()

NameError: ignored

In [50]:
feature_importances.sort_values(ascending=True).plot(kind='barh');

NameError: ignored


# Explore decision trees
install graphviz on Windows

install graphviz: pip install -U graphviz
download graphviz: https://www2.graphviz.org/Packages/stable/windows/10/msbuild/Release/Win32/
extract and copy to: C:\Program Files (x86)\Graphviz\bin
add C:\Program Files (x86)\Graphviz\bin to PATH
verify after installation: dot -V
install graphviz on macOS

install graphviz: pip install -U graphviz
sudo chown -R $(whoami) /usr/local/lib/pkgconfig /usr/local/share/info
chmod u+w /usr/local/lib/pkgconfig /usr/local/share/info
install graphviz: brew install graphviz
verify after installation: dot -V

In [51]:
clf.__dict__

NameError: ignored

In [52]:
pool = cb.Pool(X, y, cat_features=cat_features, feature_names=list(X.columns))
clf.plot_tree(tree_idx=0, pool=pool)

NameError: ignored

In [53]:
w_tree_index=widgets.BoundedIntText(
    value=0,
    min=0,
    max=clf._tree_count-1,
    step=1,
    continuous_update=True,
)
w_max_depth=widgets.BoundedIntText(
    value=2,
    min=2,
    max=10,
    step=1,
    continuous_update=True,
)
w_case_index=widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(X)-1,
    step=1,
    continuous_update=True,
)
w_n_estimators=widgets.BoundedIntText(
    value=3,
    min=1,
    max=10,
    step=1,
    continuous_update=True,
)
@interact
def plot_tree(case_index=w_case_index,
              tree_index=w_tree_index, 
              n_estimators=w_n_estimators, 
              max_depth=w_max_depth,
              save_model=[False, True],
              save_tree_img=[False, True]):
    params={
        'max_depth': max_depth, 
        'iterations': n_estimators
    }
    clf = cb.CatBoostClassifier(**params)
    fit_params={
        'cat_features': cat_features,
        'early_stopping_rounds': 10, 
        'eval_set': [(X_test, y_test)],
        'verbose': 0
    }
    clf.fit(X_train, y_train, **fit_params)
    w_tree_index.max=w_n_estimators.value-1
    print(X.iloc[[case_index]])
#     print(clf.predict(X.iloc[[case_index]]))
    print(f'proba = {clf.predict_proba(X.iloc[[case_index]])}')
    pool = cb.Pool(X, y, cat_features=cat_features, feature_names=list(X.columns))
    g=clf.plot_tree(tree_idx=tree_index, pool=pool)
#     print(g)
#     obj=clf.__dict__['_object']
#     print(obj._get_leaf_values())
    if save_model:
        leaf_indexes=clf.calc_leaf_indexes(pool)
        df_leaf_indexes=pd.DataFrame(leaf_indexes)
        df_leaf_indexes.to_csv('cb_leaf_indexes.csv')
#         print(df_leaf_indexes)
#         print(leaf_indexes)
        clf.save_model('cb_model.json', format='json', pool=pool)
    if save_tree_img:
        g.render(f'tree{tree_index}', format='svg', view=False, cleanup=True)
    return g


NameError: ignored

In [54]:
d_leaf=pd.read_csv('cb_leaf_indexes.csv', index_col=[0])
d_leaf

FileNotFoundError: ignored

In [55]:
j=json.load(open('cb_model.json'))
t=j['oblivious_trees']
JSON(t)

FileNotFoundError: ignored

In [56]:
def predict_odds_proba(case_index):
    sum_leaf_values=0
    for tree_idx, leaf_idx in enumerate(d_leaf.iloc[case_idx]):
        print(tree_idx, leaf_idx, t[tree_idx]['leaf_values'][leaf_idx])
        sum_leaf_values += t[tree_idx]['leaf_values'][leaf_idx]
    odds=np.exp(sum_leaf_values)
    p=odds/(1+odds)
    return odds, p
case_idx=0
predict_odds_proba(case_idx)

NameError: ignored

In [57]:
odds=np.exp(-0.42853556400121645-0.27604526706923704-0.19678506121229877)
p=odds/(1+odds)
print(f'odds={odds}, p={p}')

odds=0.4060147084680677, p=0.2887698869881977
